## Returns Classification using Numerous Approaches
- Data source: CRSP
- Sample period: 2000/1-2018/12
* File name: crsp_filtered2.csv

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
# pd.set_option('display.max_rows', 100, 'display.min_rows', 20)

In [23]:
plt.rcParams['figure.figsize']=(15,10)

In [24]:
crsp = pd.read_csv('crsp_filtered2.csv')
crsp

,Unnamed: 0,date,permno,prc,ret,relvol,size,prc_clipped,relvol_clipped,size_clipped,tgt_ret,tgt_label,mom3m,mom6m,mom12m,sz_cat,tgt_label2
0,12,2001-01-31,80000,20.125000,0.025478,0.052441,11.180602,20.125000,0.052441,11.180602,-0.003106,3.0,0.019481,0.226563,0.171642,5,0.0
1,13,2001-02-28,80000,20.062500,-0.003106,0.023352,11.151626,20.062500,0.023352,11.151626,0.099688,1.0,0.080537,0.183824,0.229008,5,1.0
2,14,2001-03-30,80000,22.062500,0.099688,0.072649,11.244636,22.062500,0.072649,11.244636,-0.027989,6.0,0.022293,0.163043,0.360169,5,-1.0
3,15,2001-04-30,80000,21.445000,-0.027989,0.055857,11.216248,21.445000,0.055857,11.216248,0.010026,5.0,0.096273,0.146104,0.548246,5,1.0
4,16,2001-05-31,80000,21.660000,0.010026,0.025983,11.231117,21.660000,0.025983,11.231117,-0.021237,5.0,0.068910,0.151409,0.394797,5,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313618,609240,2018-06-29,93436,342.950012,0.204474,1.252693,17.884169,129.470001,1.124182,17.391356,-0.130660,9.0,0.069891,-0.085499,-0.212605,0,-1.0
313619,609241,2018-07-31,93436,298.140015,-0.130660,1.010565,17.744599,129.470001,1.010565,17.391356,0.011806,5.0,0.166894,-0.032062,0.060222,0,1.0
313620,609242,2018-08-31,93436,301.660004,0.011806,1.625692,17.756336,129.470001,1.124182,17.391356,-0.122290,8.0,0.047097,-0.130939,-0.162293,0,-1.0
313621,609243,2018-09-28,93436,264.769989,-0.122290,1.142785,17.631655,129.470001,1.124182,17.391356,0.274011,0.0,-0.120397,0.133506,-0.115626,0,1.0


In [25]:
# create copy for use 
crsp2 = crsp.copy()

## Step: Feature Selection
#### Checking for correlation among features and testing correlation between feature and Y variable

In [26]:
#feature selection 

#checking feature correlation 
corr = crsp2.iloc[:,2:17]
corr2 = corr.corr() 
corr2  #insights - 1) remove prc, relvol, size, tgt_ret, size_clipped - too much correlation between other features; 2) tgt_ret - super high correlation with tgt_label - will overstate
#note: some correlation among different momentum features, size and prc; did not remove because correlation not too high

,permno,prc,ret,relvol,size,prc_clipped,relvol_clipped,size_clipped,tgt_ret,tgt_label,mom3m,mom6m,mom12m,sz_cat,tgt_label2
permno,1.000000,-0.007688,-0.005794,0.021004,0.061366,-0.002116,0.023692,0.060884,-0.003234,0.006260,-0.005256,-0.003051,-0.005066,0.029900,-0.002314
prc,-0.007688,1.000000,0.006345,0.020708,0.313192,0.506202,0.031349,0.301154,-0.003107,-0.009878,0.009908,0.013717,0.020935,-0.220862,-0.002086
ret,-0.005794,0.006345,1.000000,0.083804,-0.013514,0.019250,0.030337,-0.013689,0.010015,0.028435,-0.005804,0.006362,-0.002186,0.006074,-0.010484
relvol,0.021004,0.020708,0.083804,1.000000,0.152354,0.064174,0.818717,0.155663,-0.025182,0.029720,0.064084,0.088096,0.136380,-0.183620,-0.030425
size,0.061366,0.313192,-0.013514,0.152354,1.000000,0.632779,0.226533,0.998444,-0.016058,-0.009333,-0.009968,-0.009297,-0.000533,-0.874910,-0.020173
prc_clipped,-0.002116,0.506202,0.019250,0.064174,0.632779,1.000000,0.095014,0.627700,-0.009143,-0.019779,0.028446,0.039183,0.057942,-0.495857,-0.009074
relvol_clipped,0.023692,0.031349,0.030337,0.818717,0.226533,0.095014,1.000000,0.231231,-0.026625,0.030291,0.038212,0.070824,0.114113,-0.270389,-0.037335
size_clipped,0.060884,0.301154,-0.013689,0.155663,0.998444,0.627700,0.231231,1.000000,-0.016290,-0.009100,-0.010154,-0.009509,-0.000656,-0.880705,-0.020741
tgt_ret,-0.003234,-0.003107,0.010015,-0.025182,-0.016058,-0.009143,-0.026625,-0.016290,1.000000,-0.795261,0.010333,0.015075,0.017189,0.014412,0.654787
tgt_label,0.006260,-0.009878,0.028435,0.029720,-0.009333,-0.019779,0.030291,-0.009100,-0.795261,1.000000,-0.000385,-0.007081,-0.008336,0.007331,-0.726055


#### Conduct Test-Train Data Split

In [27]:
# train set: ~ 2009-12-31
# test set: 2010-01-01 ~
train = crsp2[crsp2.date < '2010-01-01']
test = crsp2[crsp2.date >= '2010-01-01']

x_cols = ['permno','ret','prc_clipped','relvol_clipped','sz_cat','mom3m','mom6m','mom12m'] # input features
x_train = train[x_cols]
y_train1 = train['tgt_label'] #using tgt_label
y_train2 = train['tgt_label2']
x_test = test[x_cols]
y_test1 = test['tgt_label']
y_test2 = test['tgt_label2']

### Standardize the training & test set

In [29]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

### ML Imports

In [30]:
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score

## Classification Process
### Logistic Regression Training


In [8]:
from sklearn.linear_model import LogisticRegression

In [10]:
# define the multinomial logistic regression model
mlr = LogisticRegression(multi_class='multinomial', solver='lbfgs')
mlr_tgt1 = mlr.fit(x_train, y_train1)
mlr_tgt2 = mlr.fit(x_train, y_train2)

In [11]:
# Regression coefficient and intercept
mlr_tgt1.coef_

array([[ 0.00392732, -0.00792271, -0.04966147,  0.11902129, -0.06070932,
        -0.00200098, -0.01410747,  0.00647922],
       [ 0.01657376,  0.01815165,  0.14118607, -0.16468738,  0.10098323,
         0.01756037, -0.00166798, -0.00200363],
       [-0.02050108, -0.01022894, -0.0915246 ,  0.04566609, -0.04027392,
        -0.01555939,  0.01577545, -0.00447559]])

In [12]:
mlr_tgt2.coef_

array([[ 0.00392732, -0.00792271, -0.04966147,  0.11902129, -0.06070932,
        -0.00200098, -0.01410747,  0.00647922],
       [ 0.01657376,  0.01815165,  0.14118607, -0.16468738,  0.10098323,
         0.01756037, -0.00166798, -0.00200363],
       [-0.02050108, -0.01022894, -0.0915246 ,  0.04566609, -0.04027392,
        -0.01555939,  0.01577545, -0.00447559]])

### Decision Trees Training

In [13]:
from sklearn.tree import DecisionTreeClassifier

In [14]:
dtree = DecisionTreeClassifier(max_depth=5, min_samples_split=2, criterion = "gini")
dtree1 = dtree.fit(x_train, y_train1)
dtree2 = dtree.fit(x_train,y_train2)

In [15]:
importances = pd.DataFrame({'feature':x_cols,'importance':np.round(dtree1.feature_importances_,3)})
importances = importances.sort_values('importance',ascending=False).set_index('feature')
importances.head(10) 


,importance
feature,
prc_clipped,0.719
relvol_clipped,0.135
mom12m,0.083
permno,0.024
ret,0.016
mom6m,0.010
sz_cat,0.007
mom3m,0.006


### Random Forest Training

In [16]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(n_estimators = 100, criterion = 'gini', max_depth = 7, min_samples_split = 125)
RF1 = RF.fit(x_train, y_train1)
RF2 = RF.fit(x_train, y_train2)



### K-Nearest Neighbors (KNN) Training

In [31]:
from sklearn.neighbors import KNeighborsClassifier

In [32]:
knn = KNeighborsClassifier()
knn1 = knn.fit(x_train, y_train1)
knn2 = knn.fit(x_train, y_train2)

### Linear Discriminant Analysis (LDA) Training

In [33]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [36]:
lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
lda_svd = LinearDiscriminantAnalysis()

lda1 = lda.fit(x_train, y_train1)
lda_svd1 = lda_svd.fit(x_train, y_train1)

lda2 = lda.fit(x_train, y_train2)
lda_svd2 = lda_svd.fit(x_train, y_train2)

### Quadratic Discriminant Analysis (QDA) Training

In [67]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [68]:
qda = QuadraticDiscriminantAnalysis()
qda1 = qda.fit(x_train, y_train1)
qda2 = qda.fit(x_train, y_train2)

### Naive Bayes (Gaussian) Training

In [81]:
from sklearn.naive_bayes import GaussianNB

In [82]:
gnb = GaussianNB()
gnb1 = gnb.fit(x_train, y_train1)
gnb2 = gnb.fit(x_train, y_train2)

## Evaluate

### Logistic Regression

In [20]:
# in-sample (train set) accuracy # higher #0.13038192211921767
mlr_tgt1.score(x_train, y_train1) 
mlr_tgt2.score(x_train, y_train2)

# out-of-sample (test set) accuracy #0.1257642533989549
mlr_tgt1.score(x_test, y_test1) 
mlr_tgt2.score(x_test, y_test2)

0.09903438117393998

0.49509591891607535

0.09316617674914586

0.49693772318140045

### Decision Trees

In [21]:
dtree1.score(x_train, y_train1) #0.1301189 
dtree2.score(x_train, y_train2) #0.1301189 
dtree1.score(x_test, y_test1) #0.12926
dtree2.score(x_test, y_test2) #0.12926

0.09693012707356706

0.49951485252685845

0.09976778611066289

0.5025571589928511

### Random Forest

In [22]:
RF1.score(x_train, y_train1) #0.1301189 
RF2.score(x_train, y_train2) #0.1301189 
RF1.score(x_test, y_test1) #0.12926
RF2.score(x_test, y_test2) #0.12926

0.09814007318128148

0.510036123028723

0.09875053493380852

0.504865266835507

### KNN

In [37]:
knn1.score(x_train, y_train1)
knn2.score(x_train, y_train2)
knn1.score(x_test, y_test1)
knn2.score(x_test, y_test2)

0.07146865245905472

0.6404238902982196

0.05231477259174553

0.4344855164478992

### LDA

In [38]:
lda1.score(x_train, y_train1)
lda_svd1.score(x_train, y_train1)
lda2.score(x_train, y_train2)
lda_svd2.score(x_train, y_train2)

lda1.score(x_test, y_test1)
lda_svd1.score(x_test, y_test1)
lda2.score(x_test, y_test2)
lda_svd2.score(x_test, y_test2)

0.0988298009141815

0.09897008452087303

0.49486211290492277

0.49504915771384483

0.09329947173094057

0.09357307721988761

0.49723237524642033

0.4974358254817912

### QDA

In [69]:
qda1.score(x_train, y_train1)
qda2.score(x_train, y_train2)
qda1.score(x_test, y_test1)
qda2.score(x_test, y_test2)

0.09231830350358308

0.48761997170947263

0.09184024245655636

0.48149655186928675

### Naive Bayes

In [83]:
gnb1.score(x_train, y_train1)
gnb2.score(x_train, y_train2)
gnb1.score(x_test, y_test1)
gnb2.score(x_test, y_test2)

0.09113758314726271

0.4887539308635625

0.08959527434211911

0.48038809886278333

## Predict

### Logistic Regression

In [24]:
# Predict
MLR1_y_pred1 = mlr_tgt1.predict(x_test)
MLR2_y_pred2 = mlr_tgt2.predict(x_test)

# Prediction accuracy (compare with the test set accuracy above) #0.1257642533989549
metrics.accuracy_score(y_test1, MLR1_y_pred1)
metrics.accuracy_score(y_test2, MLR2_y_pred2)

0.09316617674914586

0.49693772318140045

### Decision Trees

In [25]:
DR1_y_pred1 = dtree1.predict(x_test)
DR2_y_pred2 = dtree2.predict(x_test)
# Prediction accuracy
metrics.accuracy_score(y_test1, DR1_y_pred1) #0.12926105 - same as out of sample
metrics.accuracy_score(y_test2, DR2_y_pred2) #0.12926105 - same as out of sample

0.09976778611066289

0.5025571589928511

### Random Forest

In [26]:
RF1_y_pred1 = RF1.predict(x_test)
RF2_y_pred2 = RF2.predict(x_test)
# Prediction accuracy
metrics.accuracy_score(y_test1, RF1_y_pred1) #0.12926105 - same as out of sample
metrics.accuracy_score(y_test2, RF2_y_pred2) #0.12926105 - same as out of sample

0.09875053493380852

0.504865266835507

### KNN

In [39]:
knn1_y_pred1 = knn1.predict(x_test)
knn2_y_pred2 = knn2.predict(x_test)

metrics.accuracy_score(y_test1, knn1_y_pred1)
metrics.accuracy_score(y_test2, knn2_y_pred2)

0.05231477259174553

0.4344855164478992

### LDA

In [40]:
lda1_y_pred1 = lda1.predict(x_test)
lda_svd1_y_pred1 = lda_svd1.predict(x_test)

lda2_y_pred2 = lda2.predict(x_test)
lda_svd2_y_pred2 = lda_svd2.predict(x_test)

metrics.accuracy_score(y_test1, lda1_y_pred1)
metrics.accuracy_score(y_test1, lda_svd1_y_pred1)

metrics.accuracy_score(y_test2, lda2_y_pred2)
metrics.accuracy_score(y_test2, lda_svd2_y_pred2)

0.09329947173094057

0.09357307721988761

0.49723237524642033

0.4974358254817912

### QDA

In [70]:
qda1_y_pred1 = qda1.predict(x_test)
metrics.accuracy_score(y_test1, qda1_y_pred1)

qda2_y_pred2 = qda2.predict(x_test)
metrics.accuracy_score(y_test2, qda2_y_pred2)

0.09184024245655636

0.48149655186928675

### Naive Bayes

In [84]:
gnb1_y_pred1 = gnb1.predict(x_test)
metrics.accuracy_score(y_test1, gnb1_y_pred1)

gnb2_y_pred2 = gnb2.predict(x_test)
metrics.accuracy_score(y_test2, gnb2_y_pred2)

0.08959527434211911

0.48038809886278333

## Hyperparameter Tuning and fitting best model

### Logistic Regression

In [30]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=0)
n_scores1 = cross_val_score(mlr_tgt1, x_train, y_train1, scoring='accuracy', cv=cv, n_jobs=-1, verbose = 10)
n_scores2 = cross_val_score(mlr_tgt2, x_train, y_train2, scoring='accuracy', cv=cv, n_jobs=-1, verbose = 10)
n_scores1
n_scores2

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done  11 out of  30 | elapsed:    6.0s remaining:   10.3s
[Parallel(n_jobs=-1)]: Done  15 out of  30 | elapsed:    9.0s remaining:    9.0s
[Parallel(n_jobs=-1)]: Done  19 out of  30 | elapsed:    9.2s remaining:    5.3s
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:    9.8s remaining:    3.0s
[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed:   11.2s remaining:    1.2s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   11.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  11 out of  30 | elapsed:    2.8s remaining:    4.8s
[Parallel(n_jobs=-1)]: Done  15 out of  30 | elapsed:    3.4s remaining:    3.4s
[Parallel(n_jobs=-1)]: Done  19 out of  30 | elapsed:    4.5s remaining:    2.6s
[Pa

array([0.12636624, 0.12648314, 0.12935469, 0.12941314, 0.12730886,
       0.12678279, 0.12877017, 0.12707505, 0.12643208, 0.13221884,
       0.1257233 , 0.12221638, 0.13023147, 0.12602291, 0.12847791,
       0.12783493, 0.13233575, 0.1323942 , 0.12485387, 0.1322773 ,
       0.1284704 , 0.12905488, 0.12818564, 0.13110825, 0.13069909,
       0.13011457, 0.12707505, 0.12462006, 0.1255553 , 0.12748422])

array([0.49634695, 0.49383365, 0.49567454, 0.49433014, 0.49421323,
       0.49321955, 0.49468085, 0.49637596, 0.49713584, 0.49403788,
       0.49833421, 0.49827576, 0.49497311, 0.49187515, 0.49584989,
       0.49625906, 0.4907061 , 0.49386252, 0.49643442, 0.4919336 ,
       0.49634695, 0.49196329, 0.49380407, 0.49579144, 0.49649287,
       0.49275193, 0.49450549, 0.49713584, 0.49620061, 0.49485621])

In [37]:
# Create regularization penalty space
penalty = ['l1', 'l2']

# Create regularization hyperparameter space
C = np.logspace(0, 4, 10)

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty)

In [32]:
MLR1_grid = GridSearchCV(mlr_tgt1, hyperparameters, cv=10, verbose=0)

In [33]:
MLR1_BR_fit = MLR1_grid.fit(x_train, y_train1)

fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/loc

In [34]:
MLR2_grid = GridSearchCV(mlr_tgt2, hyperparameters, cv=10, verbose=0)

In [35]:
MLR2_BR_fit = MLR2_grid.fit(x_train, y_train2)

in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/

### Decision Trees

In [36]:
# Parameters to test
param_grid = {'criterion': ['gini','entropy'],'max_depth': [5,10,15], 'min_samples_split': [50, 75, 100, 125]}

DR1_grid = GridSearchCV(dtree1, param_grid, cv=3, verbose = 2)
DR1_grid.fit(x_train, y_train1)


Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] END ..criterion=gini, max_depth=5, min_samples_split=50; total time=   0.5s
[CV] END ..criterion=gini, max_depth=5, min_samples_split=50; total time=   0.5s
[CV] END ..criterion=gini, max_depth=5, min_samples_split=50; total time=   0.5s
[CV] END ..criterion=gini, max_depth=5, min_samples_split=75; total time=   0.5s
[CV] END ..criterion=gini, max_depth=5, min_samples_split=75; total time=   0.5s
[CV] END ..criterion=gini, max_depth=5, min_samples_split=75; total time=   0.5s
[CV] END .criterion=gini, max_depth=5, min_samples_split=100; total time=   0.5s
[CV] END .criterion=gini, max_depth=5, min_samples_split=100; total time=   0.5s
[CV] END .criterion=gini, max_depth=5, min_samples_split=100; total time=   0.5s
[CV] END .criterion=gini, max_depth=5, min_samples_split=125; total time=   0.5s
[CV] END .criterion=gini, max_depth=5, min_samples_split=125; total time=   0.5s
[CV] END .criterion=gini, max_depth=5, min_sampl

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(max_depth=5),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 10, 15],
                         'min_samples_split': [50, 75, 100, 125]},
             verbose=2)

In [ ]:
# Parameters to test
param_grid = {'criterion': ['gini','entropy'],'max_depth': [5,10,15], 'min_samples_split': [50, 75, 100, 125]}

DR2_grid = GridSearchCV(dtree2, param_grid, cv=3, verbose = 2)
DR2_grid.fit(x_train, y_train2)


Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] criterion=gini, max_depth=5, min_samples_split=50 ...............


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  criterion=gini, max_depth=5, min_samples_split=50, total=   0.3s
[CV] criterion=gini, max_depth=5, min_samples_split=50 ...............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV]  criterion=gini, max_depth=5, min_samples_split=50, total=   0.3s
[CV] criterion=gini, max_depth=5, min_samples_split=50 ...............
[CV]  criterion=gini, max_depth=5, min_samples_split=50, total=   0.4s
[CV] criterion=gini, max_depth=5, min_samples_split=75 ...............
[CV]  criterion=gini, max_depth=5, min_samples_split=75, total=   0.3s
[CV] criterion=gini, max_depth=5, min_samples_split=75 ...............
[CV]  criterion=gini, max_depth=5, min_samples_split=75, total=   0.3s
[CV] criterion=gini, max_depth=5, min_samples_split=75 ...............
[CV]  criterion=gini, max_depth=5, min_samples_split=75, total=   0.3s
[CV] criterion=gini, max_depth=5, min_samples_split=100 ..............
[CV]  criterion=gini, max_depth=5, min_samples_split=100, total=   0.3s
[CV] criterion=gini, max_depth=5, min_samples_split=100 ..............
[CV]  criterion=gini, max_depth=5, min_samples_split=100, total=   0.3s
[CV] criterion=gini, max_depth=5, min_samples_split=100 ..............
[CV]

[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed:   50.7s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=2,
                                              max_features='log2',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'criterion': ['gini', 'entropy'],
                    

### Random Forest

In [ ]:
param_grid1 = {'criterion' : ["gini", "entropy"],
              'max_depth' : [5, 7, 8], 
              'n_estimators': [75, 100, 150],
               'min_samples_split':[75, 125, 175]}

RF1_grid = GridSearchCV(RF1, param_grid=param_grid1, cv=3, verbose = 10)
RF1_grid.fit(x_train, y_train1)


Fitting 3 folds for each of 54 candidates, totalling 162 fits
[CV] criterion=gini, max_depth=5, min_samples_split=75, n_estimators=75 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  criterion=gini, max_depth=5, min_samples_split=75, n_estimators=75, score=0.113, total=  11.3s
[CV] criterion=gini, max_depth=5, min_samples_split=75, n_estimators=75 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.3s remaining:    0.0s


[CV]  criterion=gini, max_depth=5, min_samples_split=75, n_estimators=75, score=0.133, total=  11.1s
[CV] criterion=gini, max_depth=5, min_samples_split=75, n_estimators=75 


...

[CV]  criterion=gini, max_depth=5, min_samples_split=75, n_estimators=150, score=0.068, total=  22.6s
[CV] criterion=gini, max_depth=5, min_samples_split=125, n_estimators=75 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  2.5min remaining:    0.0s


[CV]  criterion=gini, max_depth=5, min_samples_split=125, n_estimators=75, score=0.113, total=  11.3s
[CV] criterion=gini, max_depth=5, min_samples_split=125, n_estimators=75 
[CV]  criterion=gini, max_depth=5, min_samples_split=125, n_estimators=75, score=0.132, total=  11.1s
[CV] criterion=gini, max_depth=5, min_samples_split=125, n_estimators=75 
[CV]  criterion=gini, max_depth=5, min_samples_split=125, n_estimators=75, score=0.070, total=  11.4s
[CV] criterion=gini, max_depth=5, min_samples_split=125, n_estimators=100 
[CV]  criterion=gini, max_depth=5, min_samples_split=125, n_estimators=100, score=0.113, total=  15.3s
[CV] criterion=gini, max_depth=5, min_samples_split=125, n_estimators=100 
[CV]  criterion=gini, max_depth=5, min_samples_split=125, n_estimators=100, score=0.134, total=  14.9s
[CV] criterion=gini, max_depth=5, min_samples_split=125, n_estimators=100 
[CV]  criterion=gini, max_depth=5, min_samples_split=125, n_estimators=100, score=0.073, total=  14.6s
[CV] criteri

[Parallel(n_jobs=1)]: Done 162 out of 162 | elapsed: 104.7min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=7,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=125,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              random

In [ ]:
param_grid1 = {'criterion' : ["gini", "entropy"],
              'max_depth' : [5, 7, 8], 
              'n_estimators': [75, 100, 150],
               'min_samples_split':[75, 125, 175]}

RF2_grid = GridSearchCV(RF2, param_grid=param_grid1, cv=3, verbose = 10)
RF2_grid.fit(x_train, y_train2)


### KNN

In [12]:
param_grid_knn1 = {
    'n_neighbors': [3, 5, 9, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan'],
}

knn1_grid = GridSearchCV(knn1, param_grid_knn1, cv=3)
knn1_grid.fit(x_train, y_train1)

GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': [3, 5, 9, 15],
                         'weights': ['uniform', 'distance']})

In [13]:
param_grid_knn2 = {
    'n_neighbors': [3, 5, 9, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan'],
}

knn2_grid = GridSearchCV(knn2, param_grid_knn2, cv=3)
knn2_grid.fit(x_train, y_train2)

GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': [3, 5, 9, 15],
                         'weights': ['uniform', 'distance']})

### LDA - lsqr & eigen

In [43]:
# Note: SVD does not run w/ shrinkage
param_grid_lda1 = {
    'solver': ('lsqr', 'eigen'),
    'shrinkage': np.arange(0, 1, 0.01),
}

lda1_grid = GridSearchCV(lda1, param_grid_lda1, cv=3)
lda1_grid.fit(x_train, y_train1)

GridSearchCV(cv=3,
             estimator=LinearDiscriminantAnalysis(shrinkage='auto',
                                                  solver='lsqr'),
             param_grid={'shrinkage': array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99]),
                         'solver': ('lsqr', 'eigen')})

In [44]:
# SVD does not run w/ shrinkage
param_grid_lda2 = {
    'solver': ('lsqr', 'eigen'),
    'shrinkage': np.arange(0, 1, 0.01),
}

lda2_grid = GridSearchCV(lda2, param_grid_lda2, cv=3)
lda2_grid.fit(x_train, y_train2)

GridSearchCV(cv=3,
             estimator=LinearDiscriminantAnalysis(shrinkage='auto',
                                                  solver='lsqr'),
             param_grid={'shrinkage': array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99]),
                         'solver': ('lsqr', 'eigen')})

### LDA - svd

In [45]:
param_grid_lda_svd1 = {
    'store_covariance' : (True, False),
}

lda1_svd_grid = GridSearchCV(lda_svd1, param_grid_lda_svd1, cv=3)
lda1_svd_grid.fit(x_train, y_train1)

GridSearchCV(cv=3, estimator=LinearDiscriminantAnalysis(),
             param_grid={'store_covariance': (True, False)})

In [46]:
param_grid_lda_svd2 = {
    'store_covariance' : (True, False),
}

lda2_svd_grid = GridSearchCV(lda_svd2, param_grid_lda_svd2, cv=3)
lda2_svd_grid.fit(x_train, y_train2)

GridSearchCV(cv=3, estimator=LinearDiscriminantAnalysis(),
             param_grid={'store_covariance': (True, False)})

### QDA

In [71]:
param_grid_qda1 = {
    'reg_param': (0, 0.00001, 0.0001, 0.001, 0.01, 0.1),
}

qda1_grid = GridSearchCV(qda1, param_grid_qda1, cv=3)
qda1_grid.fit(x_train, y_train1)

GridSearchCV(cv=3, estimator=QuadraticDiscriminantAnalysis(),
             param_grid={'reg_param': (0, 1e-05, 0.0001, 0.001, 0.01, 0.1)})

In [72]:
param_grid_qda2 = {
    'reg_param': (0, 0.00001, 0.0001, 0.001, 0.01, 0.1),
}

qda2_grid = GridSearchCV(qda2, param_grid_qda2, cv=3)
qda2_grid.fit(x_train, y_train2)

GridSearchCV(cv=3, estimator=QuadraticDiscriminantAnalysis(),
             param_grid={'reg_param': (0, 1e-05, 0.0001, 0.001, 0.01, 0.1)})

### Naive Bayes

In [86]:
param_grid_gnb1 = {
    'var_smoothing': (0 ,1e-9, 1e-7, 1e-5),
}

gnb1_grid = GridSearchCV(gnb1, param_grid_gnb1, cv=3)
gnb1_grid.fit(x_train, y_train1)

GridSearchCV(cv=3, estimator=GaussianNB(),
             param_grid={'var_smoothing': (0, 1e-09, 1e-07, 1e-05)})

In [87]:
param_grid_gnb2 = {
    'var_smoothing': (0 ,1e-9, 1e-7, 1e-5),
}

gnb2_grid = GridSearchCV(gnb2, param_grid_gnb2, cv=3)
gnb2_grid.fit(x_train, y_train2)

GridSearchCV(cv=3, estimator=GaussianNB(),
             param_grid={'var_smoothing': (0, 1e-09, 1e-07, 1e-05)})

### Neural Networks

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
crsp = pd.read_csv('crsp_filtered2.csv')
#read a paper on how 60-20-20 is a good way to split up and train the data. So will be doing so
train = crsp
test = crsp
x_cols = ['ret','mom3m','mom6m','mom12m'] # input features
x = crsp[x_cols].values
y = crsp['tgt_label'].values

#running the data in the first set, will resplit the data after
x_train, x_test, y_train, y_test = train_test_split(
    x ,y, train_size = 0.6, test_size=0.4, random_state=42)

x_train
x_test
y_train
y_test


In [ ]:
#Take the 40 and reinput, split into 20-20. 
x_train2, x_test2, y_train2, y_test2 = train_test_split(
    x_train, y_train, train_size = 0.6, test_size=0.4, random_state=42)

x_train2
x_test2
y_train2
y_test2

In [ ]:
#am going to run prediction on validation 
clf = MLPClassifier(hidden_layer_sizes=(32,32), 
                    activation='relu',
                    alpha=0,
                    max_iter=100,
                    verbose=True,
                    early_stopping=True,
                    validation_fraction=0.65,
                    n_iter_no_change=5
                   )
# merge back the combined datasets for training
merge_x = np.concatenate((x_train2, x_train), axis=0)
merge_y = np.concatenate((y_train2, y_train), axis=0)
merge_x_t = np.concatenate((x_test2, x_test), axis=0)
merge_y_t = np.concatenate((y_test2, y_test), axis=0)
clf = clf.fit(merge_x , merge_y)

clf.score(merge_x, merge_y) 

y_pred = clf.predict(merge_x_t)

print(y_pred)

# Prediction accuracy (compare with the test set accuracy above)
metrics.accuracy_score(merge_y_t, y_pred)

### Hyperparamater tuning (mainly from demo code)

In [ ]:
# Parameters to test
param_grid = {'alpha': [0, 1e-5, 1e-3], 
              'validation_fraction': [0.1,0.5]}

grid = GridSearchCV(clf, param_grid, cv=3)
grid.fit(merge_x, merge_y)

In [ ]:
grid.best_params_

# Best classifier
best_clf = grid.best_estimator_

# in-sample
best_clf.score(merge_x, merge_y)

# out-of-sample
best_clf.score(merge_x_t, merge_y_t)

y_pred = best_clf.predict(merge_x_t)
metrics.accuracy_score(merge_y_t, y_pred)

## Best Classifiers

### Logistic Regression

In [ ]:
# Selected parameters
MLR1_grid.best_params_

# Best classifier
MLR1_BR_clf = MLR1_grid.best_estimator_

{'C': 7.742636826811269, 'penalty': 'l2'}

In [ ]:
# Selected parameters
MLR2_grid.best_params_

# Best classifier
MLR2_BR_clf = MLR2_grid.best_estimator_

{'C': 2.7825594022071245, 'penalty': 'l2'}

### Decision Trees

In [ ]:
# Selected parameters
DR1_grid.best_params_

# Best classifier
DR1_BR_clf = DR1_grid.best_estimator_

{'criterion': 'entropy', 'max_depth': 5, 'min_samples_split': 100}

In [ ]:
# Selected parameters
DR2_grid.best_params_

# Best classifier
DR2_BR_clf = DR2_grid.best_estimator_

{'criterion': 'entropy', 'max_depth': 5, 'min_samples_split': 125}

### Random Forest

In [ ]:
# Selected parameters
RF1_grid.best_params_

# Best classifier
RF1_BR_clf = RF1_grid.best_estimator_

# Selected parameters
RF2_grid.best_params_

# Best classifier
RF2_BR_clf = RF2_grid.best_estimator_

### KNN

In [14]:
knn1_grid.best_params_
knn1_BR_clf = knn1_grid.best_estimator_

knn2_grid.best_params_
knn2_BR_clf = knn2_grid.best_estimator_

{'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'uniform'}

{'metric': 'manhattan', 'n_neighbors': 15, 'weights': 'uniform'}

### LDA

In [48]:
lda1_grid.best_params_
lda1_BR_clf = knn1_grid.best_estimator_

lda1_svd_grid.best_params_
lda_svd1_BR_clf = lda1_svd_grid.best_estimator_

lda2_grid.best_params_
lda2_BR_clf = knn2_grid.best_estimator_

lda2_svd_grid.best_params_
lda_svd2_BR_clf = lda2_svd_grid.best_estimator_

{'shrinkage': 0.78, 'solver': 'lsqr'}

{'store_covariance': True}

{'shrinkage': 0.12, 'solver': 'lsqr'}

{'store_covariance': True}

### QDA

In [73]:
qda1_grid.best_params_
qda1_BR_clf = lda1_grid.best_estimator_

qda2_grid.best_params_
qda2_BR_clf = lda2_grid.best_estimator_

{'reg_param': 0.001}

{'reg_param': 0.1}

### Naive Bayes

In [88]:
gnb1_grid.best_params_
gnb1_BR_clf = gnb1_grid.best_estimator_

gnb2_grid.best_params_
gnb2_BR_clf = gnb2_grid.best_estimator_

{'var_smoothing': 0}

{'var_smoothing': 0}

## Evaluate

### Logistic Regression

In [ ]:
# in-sample
MLR1_BR_clf.score(x_train, y_train1)

# out-of-sample
MLR1_BR_clf.score(x_test, y_test1)

0.12900246665341766

0.12821574143579742

In [ ]:
# in-sample
MLR2_BR_clf.score(x_train, y_train2)

# out-of-sample
MLR2_BR_clf.score(x_test, y_test2)

0.49509591891607535

0.49693772318140045

### Decision Trees

In [ ]:
# in-sample
DR1_BR_clf.score(x_train, y_train1)

# out-of-sample
DR1_BR_clf.score(x_test, y_test1)

0.12942916262377105

0.12475007190913491

In [ ]:
# in-sample
DR2_BR_clf.score(x_train, y_train2)

# out-of-sample
DR2_BR_clf.score(x_test, y_test2)

0.497521656281783

0.4950786089616321

### Random Forest

In [ ]:
# in-sample
RF1_BR_clf.score(x_train, y_train1)

# out-of-sample
RF1_BR_clf.score(x_test, y_test1)

# in-sample
RF2_BR_clf.score(x_train, y_train2)

# out-of-sample
RF2_BR_clf.score(x_test, y_test2)

### KNN

In [15]:
# in-sample
knn1_BR_clf.score(x_train, y_train1)

# out-of-sample
knn1_BR_clf.score(x_test, y_test1)

# in-sample
knn2_BR_clf.score(x_train, y_train2)

# out-of-sample
knn2_BR_clf.score(x_test, y_test2)

0.3755625957143358

0.10894409327842515

0.5747010205632387

0.45293634813843037

### LDA - lsqr & eigen

In [75]:
# in-sample
lda1_BR_clf.score(x_train, y_train1)

# out-of-sample
lda1_BR_clf.score(x_test, y_test1)

# in-sample
lda2_BR_clf.score(x_train, y_train2)

# out-of-sample
lda2_BR_clf.score(x_test, y_test2)

0.07627921113851838

0.07837744929529047

0.4939327340105914

0.5075381819967588

### LDA - svd

In [74]:
# in-sample
lda_svd1_BR_clf.score(x_train, y_train1)

# out-of-sample
lda_svd1_BR_clf.score(x_test, y_test1)

# in-sample
lda_svd2_BR_clf.score(x_train, y_train2)

# out-of-sample
lda_svd2_BR_clf.score(x_test, y_test2)

0.12935317567014648

0.12798422909899607

0.49504915771384483

0.4974358254817912

### QDA

In [76]:
# in-sample
qda1_BR_clf.score(x_train, y_train1)

# out-of-sample
qda1_BR_clf.score(x_test, y_test1)

# in-sample
qda2_BR_clf.score(x_train, y_train2)

# out-of-sample
qda2_BR_clf.score(x_test, y_test2)

0.1296220525829719

0.1288541542433405

0.4946867583965584

0.49702190948569186

### Naive Bayes

In [89]:
# in-sample
gnb1_BR_clf.score(x_train, y_train1)

# out-of-sample
gnb1_BR_clf.score(x_test, y_test1)

# in-sample
gnb2_BR_clf.score(x_train, y_train2)

# out-of-sample
gnb2_BR_clf.score(x_test, y_test2)

0.12447247518733706

0.12351533944619443

0.4887539308635625

0.48038809886278333

## Prediction

### Logistic Regression

In [ ]:
MLR1_y_pred1 = MLR1_BR_clf.predict(x_test)
metrics.accuracy_score(y_test1, MLR1_y_pred1)

0.12821574143579742

In [ ]:
MLR2_y_pred2 = MLR2_BR_clf.predict(x_test)
metrics.accuracy_score(y_test2, MLR2_y_pred2)

0.49693772318140045

### Decision Trees

In [ ]:
DR1_y_pred1 = DR1_BR_clf.predict(x_test)
metrics.accuracy_score(y_test1, DR1_y_pred1)

0.12475007190913491

In [ ]:
DR2_y_pred2 = DR2_BR_clf.predict(x_test)
metrics.accuracy_score(y_test2, DR2_y_pred2)

0.4950786089616321

### Random Forest

In [ ]:
RF1_y_pred1 = RF1_BR_clf.predict(x_test)
metrics.accuracy_score(y_test1, RF1_y_pred1)

RF2_y_pred2 = RF2_BR_clf.predict(x_test)
metrics.accuracy_score(y_test2, RF2_y_pred2)

### KNN

In [16]:
knn1_y_pred1 = knn1_BR_clf.predict(x_test)
metrics.accuracy_score(y_test1, knn1_y_pred1)

knn2_y_pred2 = knn2_BR_clf.predict(x_test)
metrics.accuracy_score(y_test2, knn2_y_pred2)

0.10894409327842515

0.45293634813843037

### LDA - lsqr & eigen

In [52]:
lda1_y_pred1 = lda1_BR_clf.predict(x_test)
metrics.accuracy_score(y_test1, lda1_y_pred1)

lda2_y_pred2 = lda2_BR_clf.predict(x_test)
metrics.accuracy_score(y_test2, lda2_y_pred2)

0.07837744929529047

0.5075381819967588

### LDA - svd

In [53]:
lda_svd1_y_pred1 = lda_svd1_BR_clf.predict(x_test)
metrics.accuracy_score(y_test1, lda_svd1_y_pred1)

lda_svd2_y_pred1 = lda_svd2_BR_clf.predict(x_test)
metrics.accuracy_score(y_test2, lda_svd2_y_pred2)

0.12798422909899607

0.4974358254817912

### QDA

In [77]:
qda1_y_pred1 = qda1_BR_clf.predict(x_test)
metrics.accuracy_score(y_test1, qda1_y_pred1)

qda2_y_pred2 = qda2_BR_clf.predict(x_test)
metrics.accuracy_score(y_test2, qda2_y_pred2)

0.1288541542433405

0.49702190948569186

### Naive Bayes

In [90]:
gnb1_y_pred1 = gnb1_BR_clf.predict(x_test)
metrics.accuracy_score(y_test1, gnb1_y_pred1)

gnb2_y_pred2 = gnb2_BR_clf.predict(x_test)
metrics.accuracy_score(y_test2, gnb2_y_pred2)

0.12351533944619443

0.48038809886278333

## Testing

### Logistic Regression

In [ ]:
print("Accuracy Score TGT1:", accuracy_score(y_test1, MLR1_y_pred1) )
print("Accuracy Score TGT2:", accuracy_score(y_test2, MLR2_y_pred2) )
print("Precision TGT1:", precision_score(y_test1, MLR1_y_pred1, average = 'weighted') )
print("Precision TGT2:", precision_score(y_test2, MLR2_y_pred2, average = 'weighted') )
print("Recall TGT1:", recall_score(y_test1, MLR1_y_pred1, average = 'weighted'))
print("Recall TGT2:", recall_score(y_test2, MLR2_y_pred2, average = 'weighted'))
print("F1 TGT1:", f1_score(y_test1, MLR1_y_pred1, average = 'weighted'))
print("F1 TGT2:", f1_score(y_test2, MLR2_y_pred2, average = 'weighted'))
print("\n classification report TGT1:\n", classification_report(y_test1, MLR1_y_pred1))
print("\n classification report TGT2:\n", classification_report(y_test2, MLR2_y_pred2))

### Decision Tree

In [ ]:
print("Accuracy Score TGT1:", accuracy_score(y_test1, DR1_y_pred1) )
print("Accuracy Score TGT2:", accuracy_score(y_test2, DR2_y_pred2) )
print("Precision TGT1:", precision_score(y_test1, DR1_y_pred1, average = 'weighted') )
print("Precision TGT2:", precision_score(y_test2, DR2_y_pred2, average = 'weighted') )
print("Recall TGT1:", recall_score(y_test1, DR1_y_pred1, average = 'weighted'))
print("Recall TGT2:", recall_score(y_test2, DR2_y_pred2, average = 'weighted'))
print("F1 TGT1:", f1_score(y_test1, DR1_y_pred1, average = 'weighted'))
print("F1 TGT2:", f1_score(y_test2, DR2_y_pred2, average = 'weighted'))
print("\n classification report TGT1:\n", classification_report(y_test1, DR1_y_pred1))
print("\n classification report TGT2:\n", classification_report(y_test2, DR2_y_pred2))

### Random Forest

In [ ]:
print("Accuracy Score TGT1:", accuracy_score(y_test1, RF1_y_pred1) )
print("Accuracy Score TGT2:", accuracy_score(y_test2, RF2_y_pred2) )
print("Precision TGT1:", precision_score(y_test1, RF1_y_pred1, average = 'weighted') )
print("Precision TGT2:", precision_score(y_test2, RF2_y_pred2, average = 'weighted') )
print("Recall TGT1:", recall_score(y_test1, RF1_y_pred1, average = 'weighted'))
print("Recall TGT2:", recall_score(y_test2, RF2_y_pred2, average = 'weighted'))
print("F1 TGT1:", f1_score(y_test1, RF1_y_pred1, average = 'weighted'))
print("F1 TGT2:", f1_score(y_test2, RF2_y_pred2, average = 'weighted'))
print("\n classification report TGT1:\n", classification_report(y_test1, RF1_y_pred1))
print("\n classification report TGT2:\n", classification_report(y_test2, RF2_y_pred2))

### KNN

In [17]:
print("Accuracy Score TGT1:", accuracy_score(y_test1, knn1_y_pred1) )
print("Accuracy Score TGT2:", accuracy_score(y_test2, knn2_y_pred2) )
print("Precision TGT1:", precision_score(y_test1, knn1_y_pred1, average = 'weighted') )
print("Precision TGT2:", precision_score(y_test2, knn2_y_pred2, average = 'weighted') )
print("Recall TGT1:", recall_score(y_test1, knn1_y_pred1, average = 'weighted'))
print("Recall TGT2:", recall_score(y_test2, knn2_y_pred2, average = 'weighted'))
print("F1 TGT1:", f1_score(y_test1, knn1_y_pred1, average = 'weighted'))
print("F1 TGT2:", f1_score(y_test2, knn2_y_pred2, average = 'weighted'))
print("\n classification report TGT1:\n", classification_report(y_test1, knn1_y_pred1))
print("\n classification report TGT2:\n", classification_report(y_test2, knn2_y_pred2))

Accuracy Score TGT1: 0.10894409327842515
Accuracy Score TGT2: 0.4411993742151381
Precision TGT1: 0.11183709448435088
Precision TGT2: 0.4268213606176114
Recall TGT1: 0.10894409327842515
Recall TGT2: 0.45293634813843037
F1 TGT1: 0.10589029819228506
F1 TGT2: 0.43285934891256234

 classification report TGT1:
               precision    recall  f1-score   support

         0.0       0.09      0.21      0.13     11158
         1.0       0.11      0.17      0.13     15253
         2.0       0.11      0.14      0.13     16200
         3.0       0.12      0.11      0.12     16447
         4.0       0.13      0.09      0.11     16222
         5.0       0.13      0.08      0.10     15831
         6.0       0.11      0.08      0.09     15429
         7.0       0.10      0.07      0.08     14489
         8.0       0.10      0.08      0.09     12714
         9.0       0.09      0.08      0.08      8798

    accuracy                           0.11    142541
   macro avg       0.11      0.11      0.10

### LDA - lsqr & eigen

In [64]:
print("Accuracy Score TGT1:", accuracy_score(y_test1, lda1_y_pred1) )
print("Accuracy Score TGT2:", accuracy_score(y_test2, lda2_y_pred2) )
print("Precision TGT1:", precision_score(y_test1, lda1_y_pred1, average = 'weighted') )
print("Precision TGT2:", precision_score(y_test2, lda2_y_pred2, average = 'weighted') )
print("Recall TGT1:", recall_score(y_test1, lda1_y_pred1, average = 'weighted'))
print("Recall TGT2:", recall_score(y_test2, lda2_y_pred2, average = 'weighted'))
print("F1 TGT1:", f1_score(y_test1, lda1_y_pred1, average = 'weighted'))
print("F1 TGT2:", f1_score(y_test2, lda2_y_pred2, average = 'weighted'))
print("\n classification report TGT1:\n", classification_report(y_test1, lda1_y_pred1))
print("\n classification report TGT2:\n", classification_report(y_test2, lda2_y_pred2))

Accuracy Score TGT1: 0.07837744929529047
Accuracy Score TGT2: 0.5075381819967588
Precision TGT1: 0.018357675048675758
Precision TGT2: 0.5486112238967392
Recall TGT1: 0.07837744929529047
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Recall TGT2: 0.5075381819967588
F1 TGT1: 0.01286594498427558
F1 TGT2: 0.3417706095399292
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are

### LDA - svd

In [65]:
print("Accuracy Score TGT1:", accuracy_score(y_test1, lda_svd1_y_pred1) )
print("Accuracy Score TGT2:", accuracy_score(y_test2, lda_svd2_y_pred2) )
print("Precision TGT1:", precision_score(y_test1, lda_svd1_y_pred1, average = 'weighted') )
print("Precision TGT2:", precision_score(y_test2, lda_svd2_y_pred2, average = 'weighted') )
print("Recall TGT1:", recall_score(y_test1, lda_svd1_y_pred1, average = 'weighted'))
print("Recall TGT2:", recall_score(y_test2, lda_svd2_y_pred2, average = 'weighted'))
print("F1 TGT1:", f1_score(y_test1, lda_svd1_y_pred1, average = 'weighted'))
print("F1 TGT2:", f1_score(y_test2, lda_svd2_y_pred2, average = 'weighted'))
print("\n classification report TGT1:\n", classification_report(y_test1, lda_svd1_y_pred1))
print("\n classification report TGT2:\n", classification_report(y_test2, lda_svd2_y_pred2))

Accuracy Score TGT1: 0.12798422909899607
Accuracy Score TGT2: 0.4974358254817912
Precision TGT1: 0.12284390897824418
Precision TGT2: 0.4338602704017685
Recall TGT1: 0.12798422909899607
Recall TGT2: 0.4974358254817912
F1 TGT1: 0.10067244874579065
F1 TGT2: 0.4006692399409426

 classification report TGT1:
               precision    recall  f1-score   support

         0.0       0.16      0.11      0.13     11158
         1.0       0.12      0.09      0.11     15253
         2.0       0.12      0.17      0.14     16200
         3.0       0.13      0.20      0.16     16447
         4.0       0.13      0.44      0.20     16222
         5.0       0.10      0.01      0.02     15831
         6.0       0.10      0.03      0.05     15429
         7.0       0.11      0.01      0.01     14489
         8.0       0.12      0.03      0.05     12714
         9.0       0.16      0.13      0.14      8798

    accuracy                           0.13    142541
   macro avg       0.13      0.12      0.10  

### QDA

In [80]:
print("Accuracy Score TGT1:", accuracy_score(y_test1, qda1_y_pred1) )
print("Accuracy Score TGT2:", accuracy_score(y_test2, qda2_y_pred2) )
print("Precision TGT1:", precision_score(y_test1, qda1_y_pred1, average = 'weighted') )
print("Precision TGT2:", precision_score(y_test2, qda2_y_pred2, average = 'weighted') )
print("Recall TGT1:", recall_score(y_test1, qda1_y_pred1, average = 'weighted'))
print("Recall TGT2:", recall_score(y_test2, qda2_y_pred2, average = 'weighted'))
print("F1 TGT1:", f1_score(y_test1, qda1_y_pred1, average = 'weighted'))
print("F1 TGT2:", f1_score(y_test2, qda2_y_pred2, average = 'weighted'))
print("\n classification report TGT1:\n", classification_report(y_test1, qda1_y_pred1))
print("\n classification report TGT2:\n", classification_report(y_test2, qda2_y_pred2))

Accuracy Score TGT1: 0.1288541542433405
Accuracy Score TGT2: 0.49702190948569186
Precision TGT1: 0.12355071906801725
Precision TGT2: 0.41492783728056015
Recall TGT1: 0.1288541542433405
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Recall TGT2: 0.49702190948569186
F1 TGT1: 0.1081876813380111
F1 TGT2: 0.4023872998924251

 classification report TGT1:
               precision    recall  f1-score   support

         0.0       0.14      0.15      0.15     11158
         1.0       0.13      0.09      0.10     15253
         2.0       0.12      0.17      0.14     16200
         3.0       0.13      0.29      0.18     16447
         4.0       0.13      0.30      0.18     16222
         5.0       0.11      0.03      0.04     15831
         6.0      

### Naive Bayes

In [91]:
print("Accuracy Score TGT1:", accuracy_score(y_test1, gnb1_y_pred1) )
print("Accuracy Score TGT2:", accuracy_score(y_test2, gnb2_y_pred2) )
print("Precision TGT1:", precision_score(y_test1, gnb1_y_pred1, average = 'weighted') )
print("Precision TGT2:", precision_score(y_test2, gnb2_y_pred2, average = 'weighted') )
print("Recall TGT1:", recall_score(y_test1, gnb1_y_pred1, average = 'weighted'))
print("Recall TGT2:", recall_score(y_test2, gnb2_y_pred2, average = 'weighted'))
print("F1 TGT1:", f1_score(y_test1, gnb1_y_pred1, average = 'weighted'))
print("F1 TGT2:", f1_score(y_test2, gnb2_y_pred2, average = 'weighted'))
print("\n classification report TGT1:\n", classification_report(y_test1, gnb1_y_pred1))
print("\n classification report TGT2:\n", classification_report(y_test2, gnb2_y_pred2))

Accuracy Score TGT1: 0.12351533944619443
Accuracy Score TGT2: 0.48038809886278333
Precision TGT1: 0.12055495715324052
Precision TGT2: 0.4331366053269798
Recall TGT1: 0.12351533944619443
Recall TGT2: 0.48038809886278333
F1 TGT1: 0.07455037336058991
F1 TGT2: 0.40597153997299135

 classification report TGT1:
               precision    recall  f1-score   support

         0.0       0.19      0.04      0.07     11158
         1.0       0.13      0.03      0.05     15253
         2.0       0.10      0.01      0.02     16200
         3.0       0.13      0.64      0.21     16447
         4.0       0.09      0.03      0.04     16222
         5.0       0.11      0.25      0.15     15831
         6.0       0.09      0.02      0.03     15429
         7.0       0.11      0.01      0.02     14489
         8.0       0.14      0.01      0.02     12714
         9.0       0.17      0.11      0.13      8798

    accuracy                           0.12    142541
   macro avg       0.12      0.12      0.0